In [1]:
#Instalamos mongo y cargamos la base de datos

!pip install pymongo

from pymongo import MongoClient

# Parametros de conexion
usuario = "lab_user"
password = "bdnr2025"
host = "bdnr-2025-auriello-moli.u17itbj.mongodb.net"
db_name = "tarea_final"

# URI de conexión a Mongodb atlas
uri = f"mongodb+srv://{usuario}:{password}@{host}/{db_name}?retryWrites=true&w=majority&appName=bdnr-2025-auriello-molina"

# creamos el cliente de conexión
client = MongoClient(uri)

# accedemos a la base de datos
db = client[db_name]

# Probar conexión listando las colecciones
print("Conexion exitosa. Colecciones disponibles:")
print(db.list_collection_names())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 9.9 MB/s eta 0:00:00
Conexion exitosa. Colecciones disponibles:
['video_views', 'dim_media', 'dim_location', 'users', 'dim_plataform', 'clients', 'media', 'fact_vod_uu_details', 'dim_date', 'dim_device', 'dim_client', 'dim_partner']


# **Exploración inicial del dataset**

In [ ]:
##Exploración inicial de la base de datos

# Listar todas las colecciones y mostrar cuántos documentos hay en cada una

print("Colecciones disponibles en la base de datos y su tamaño:")
for collection_name in db.list_collection_names():
    count = db[collection_name].count_documents({})
    print(f" - {collection_name}: {count} documentos")



Colecciones disponibles en la base de datos y su tamaño:
 - video_views: 215347 documentos
 - dim_media: 70058 documentos
 - dim_location: 342 documentos
 - users: 34769 documentos
 - dim_plataform: 13 documentos
 - clients: 6 documentos
 - media: 70058 documentos
 - fact_vod_uu_details: 215347 documentos
 - dim_date: 10847 documentos
 - dim_device: 23547 documentos
 - dim_client: 3 documentos
 - dim_partner: 74858 documentos


In [ ]:
from pprint import pprint

# Mostrar un documento de ejemplo de cada colección para ver como estan estructurados (tipos de datos, campos embedded, keys)
for collection_name in db.list_collection_names():
    print(f"\n📄 Documento ejemplo de la colección: {collection_name}")
    doc = db[collection_name].find_one()
    pprint(doc)



📄 Documento ejemplo de la colección: video_views
{'_id': ObjectId('683dfda82d64dba076265475'),
 'client': {'client_id': 'tg', 'client_name': 'Toon Goggles'},
 'date': {'date_code': 20250509, 'full_date': '2025-05-09'},
 'device': {'device_type_id': 'tv', 'device_type_name': 'TV'},
 'location': {'country_code': 'US', 'country_name': 'United States'},
 'media': {'category_name': 'N/A',
           'media_id': 284074,
           'media_name': 'Suzy and The Pothole',
           'video_type': 'Episode'},
 'partner': {'partner_id': 'viziosmartcast', 'partner_name': 'Vizio Smartcast'},
 'platform': {'platform_id': 'html5', 'platform_name': 'HTML5'},
 'seconds_elapsed': 358,
 'user_id': '4c8f120a1d08d1d94ec21dabbc0b3549f26ab431'}

📄 Documento ejemplo de la colección: dim_media
{'_id': ObjectId('68333a654e58b8a122cf5d52'),
 'category_name': 'Unknown',
 'id': 352389,
 'media_id': -99,
 'media_name': 'Unknown',
 'video_type': 'Unknown'}

📄 Documento ejemplo de la colección: dim_location
{'_id': O

# **Transformaciones**

Agregar campo countries a collección de clientes

In [ ]:
#Hacemos el aggregate de paises para un client id
pipeline = [
      {
          "$group": {
              "_id": "$client_id",
              "countries": { "$addToSet": "$country_name" }  # Use $country_code if needed
          }
      }
  ]

results = db.fact_vod_uu_details_mod.aggregate(pipeline)

# Step 2: Listamos los paises en los que se utiliza cada servicio
for doc in results:
    client_id = doc["_id"]
    countries = doc["countries"]
    print(client_id, countries)

tg ['India', 'Mozambique', 'Dominica', 'Saint Lucia', 'China', 'Antigua and Barbuda', 'Liberia', 'Egypt', 'Puerto Rico', 'Tanzania', 'Togo', 'Palestinian Territories', 'Russia', 'Saint Vincent and the Grenadines', 'Réunion', 'Syria', 'Guadeloupe', 'Niger', 'Sri Lanka', 'Netherlands', 'Maldives', 'Chile', 'New Zealand', 'Kenya', 'Oman', 'South Sudan', 'Macedonia', 'Peru', 'Côte d’Ivoire', 'Nepal', 'Paraguay', 'Belarus', 'Panama', 'Qatar', 'Mayotte', 'Hungary', 'French Guiana', 'Slovakia', 'Georgia', 'Lebanon', 'United Arab Emirates', 'Philippines', 'Kazakhstan', 'Estonia', 'Trinidad and Tobago', 'Tajikistan', 'Turks and Caicos Islands', 'Uruguay', 'Palau', 'Greenland', 'Laos', 'Azerbaijan', 'Bolivia', 'Gambia', 'Brazil', 'Ethiopia', 'Zambia', 'Bosnia and Herzegovina', 'Barbados', 'Nicaragua', 'Austria', 'Libya', 'Bahamas', 'Norway', 'Martinique', 'Mauritania', 'Afghanistan', 'Poland', 'Equatorial Guinea', 'Congo - Brazzaville', 'Kyrgyzstan', 'Mauritius', 'United States', 'Cambodia', 'Gr

Creamos una función para obtener la lista de paises y asignarselos a la colección de clientes

In [ ]:
def update_client_countries():
    # Step 1: Aggregate countries per client_id
    pipeline = [
      {
          "$group": {
              "_id": "$client_id",
              "countries": { "$addToSet": "$country_name" }  # Use $country_code if needed
          }
      }
    ]

    results = db.fact_vod_uu_details_mod.aggregate(pipeline)

    # Step 2: Update each client with the countries list
    for doc in results:
        client_id = doc["_id"]
        countries = doc["countries"]

        result = db.dim_client.update_one(
            { "client_id": client_id },           # Adjust this if your key is different
            { "$set": { "countries": countries } },
            upsert=False                          # Change to True if you want to insert if missing
        )

        print(f"Updated client_id={client_id}, matched={result.matched_count}, modified={result.modified_count}")

#    client.close()

if __name__ == "__main__":
    update_client_countries()


Updated client_id=tg, matched=1, modified=1


Agregar los dispositivos

In [ ]:
def update_client_partners():
    pipeline = [
        {
            "$group": {
                "_id": "$client_id",
                "partners": {
                    "$addToSet": {
                        "partner_id": "$partner_id",
                        "partner_name": "$partner_name"
                    }
                }
            }
        }
    ]

    results = db.fact_vod_uu_details.aggregate(pipeline)

    for doc in results:
        client_id = doc["_id"]
        partners = doc["partners"]

        result = db.dim_client.update_one(
            { "client_id": client_id },
            { "$set": { "partners": partners } },
            upsert=True
        )

        print(f"Updated client_id={client_id}, matched={result.matched_count}, modified={result.modified_count}")


if __name__ == "__main__":
    update_client_partners()


Updated client_id=tg, matched=1, modified=1


Agregaremos Paises a Users #Agregar colleccion users

In [ ]:
def update_users_countries():
    # Step 1: Aggregate countries per client_id
    pipeline = [
      {
          "$group": {
              "_id": "$client_id",
              "countries": { "$addToSet": "$country_name" }  # Use $country_code if needed
          }
      }
    ]

    results = db.fact_vod_uu_details.aggregate(pipeline)

    # Step 2: Update each client with the countries list
    for doc in results:
        client_id = doc["_id"]
        countries = doc["countries"]

        result = db.dim_client.update_one(
            { "client_id": client_id },           # Adjust this if your key is different
            { "$set": { "countries": countries } },
            upsert=False                          # Change to True if you want to insert if missing
        )

        print(f"Updated client_id={client_id}, matched={result.matched_count}, modified={result.modified_count}")

#    client.close()

if __name__ == "__main__":
    update_client_countries()


Updated client_id=tg, matched=1, modified=1


## **Creacion de las colecciones de nuestra base de datos no relacional - documental**



### **Coleccion clients**

Copiamos dim_clients que fue modificado para contemplar el diseno no relacional a la coleccion clients, y luego lo droppeamos. Despues, procedemos a subir el archivo original de dim_clients de la base de datos relacional.

In [ ]:
# Copio todos los docuemntos en dim_client a clients
db["clients"].insert_many(db["dim_client"].find({}))

# Drop dim_client modificada
db["dim_client"].drop()

print("dim_client que fue modificado se renombra a clients")


dim_client que fue modificado se renombra a clients


In [ ]:
#subo de nuevo el archivio de dim_client original
from google.colab import files

# Abre diálogo para subir archivos (elige dim_client.json)
uploaded = files.upload()


TypeError: 'NoneType' object is not subscriptable

In [ ]:
import json

# Nombre del archivo
json_filename = 'dim_client.json'

# Abrimos el archivo y cargamos los datos
with open(json_filename) as f:
    data = json.load(f)

# Restauramos la colección original
db["dim_client"].drop()  # Aseguramos que esté vacía antes de insertar
db["dim_client"].insert_many(data)

print("La colección 'dim_client' ha sido restaurada con éxito desde el archivo original.")


**Creacion de indices para acelerar consultas**

In [ ]:
# Buscar cliente por ID o nombre
db.clients.create_index("client_id")
db.clients.create_index("client_name")

# Búsqueda por partner dentro del array (para match específicos - ejemplo: quiero saber si mi app esta en Roku, o en Amazon)
db.clients.create_index("partners.partner_id")


### **Colleccion video_views**

Esta va a ser la coleccion principal de nuestro modelo no relacional

In [ ]:
#db["video_views"].drop()

Creamos la coleccion principal video_views a partir de fact_vod_uu_details directamente. Primero realizamos nuevamente la coneccion a MongoDB y seteamos en la URL un timeout mas alto para que no se nos desconecte de la instancia.

en este codigo crearemos video_views haciendo:
- leemos cada documento de fact_vod_uu_details
- extraemos y simplificamos/reestructuramos los datos a un formato mas limpio (desnormalizado)
- hacemos append a los documentos nuevos a la lista video_views_docs

Hacemos una reconeccion antes del insert (para evitar timeout)

Insertamos los documentos de la nueva lista en baches de a 1000 a travs de insert_many (son como 250 batches)

In [ ]:
# Ensure pymongo is installed
!pip install pymongo

from pymongo import MongoClient
from pymongo.errors import BulkWriteError

# Connection parameters
usuario = "lab_user"
password = "bdnr2025"
host = "bdnr-2025-auriello-moli.u17itbj.mongodb.net"
db_name = "tarea_final"

# MongoDB Atlas URI with increased timeouts
uri = f"mongodb+srv://{usuario}:{password}@{host}/{db_name}?retryWrites=true&w=majority&appName=bdnr-2025-auriello-molina&socketTimeoutMS=60000&connectTimeoutMS=60000"

# Connect to MongoDB
client = MongoClient(uri)
db = client[db_name]

# Optional: Test connection
print("Conexión exitosa. Colecciones disponibles:")
print(db.list_collection_names())

# Create transformed documents
video_views_docs = []

for doc in db["fact_vod_uu_details"].find({}):
    video_views_docs.append({
        "user_id": doc.get("uuid_hashvalue"),
        "date": {
            "date_code": doc.get("date_code"),
            "full_date": doc.get("full_date")
        },
        "seconds_elapsed": doc.get("seconds_watched"),
        "client": {
            "client_id": doc.get("client_id"),
            "client_name": doc.get("client_name")
        },
        "device": {
            "device_type_id": doc.get("device_type_id"),
            "device_type_name": doc.get("device_type_name")
        },
        "location": {
            "country_code": doc.get("country_code"),
            "country_name": doc.get("country_name")
        },
        "partner": {
            "partner_id": doc.get("partner_id"),
            "partner_name": doc.get("partner_name")
        },
        "platform": {
            "platform_id": doc.get("platform_id"),
            "platform_name": doc.get("platform_name")
        },
        "media": {
            "media_id": doc.get("media_id"),
            "media_name": doc.get("media_name"),
            "category_name": doc.get("category_name"),
            "video_type": doc.get("video_type")
        }
    })

print(f"Total documents to insert: {len(video_views_docs)}")

# Reconnect before inserting (just in case of idle timeout)
client = MongoClient(uri)
db = client[db_name]

# Insert documents in batches
batch_size = 1000
for i in range(0, len(video_views_docs), batch_size):
    batch = video_views_docs[i:i + batch_size]
    try:
        db["video_views"].insert_many(batch)
        print(f"Batch {i // batch_size + 1} inserted successfully.")
    except BulkWriteError as bwe:
        print(f"Error inserting batch {i // batch_size + 1}: {bwe.details}")

print("Colección 'video_views' creada con éxito en lotes.")


Conexión exitosa. Colecciones disponibles:
['dim_media', 'dim_location', 'dim_plataform', 'clients', 'fact_vod_uu_details', 'dim_date', 'dim_device', 'dim_client', 'dim_partner']
Total documents to insert: 215347
Batch 1 inserted successfully.
Batch 2 inserted successfully.
Batch 3 inserted successfully.
Batch 4 inserted successfully.
Batch 5 inserted successfully.
Batch 6 inserted successfully.
Batch 7 inserted successfully.
Batch 8 inserted successfully.
Batch 9 inserted successfully.
Batch 10 inserted successfully.
Batch 11 inserted successfully.
Batch 12 inserted successfully.
Batch 13 inserted successfully.
Batch 14 inserted successfully.
Batch 15 inserted successfully.
Batch 16 inserted successfully.
Batch 17 inserted successfully.
Batch 18 inserted successfully.
Batch 19 inserted successfully.
Batch 20 inserted successfully.
Batch 21 inserted successfully.
Batch 22 inserted successfully.
Batch 23 inserted successfully.
Batch 24 inserted successfully.
Batch 25 inserted successful

In [ ]:
from pprint import pprint

# Traemos los primeros 3 documentos de la colección para ver que quedo bien (check que haya uuid_hashvalue, sessions, y metadata)
print("Primeros 3 documentos de 'video_views':\n")

for doc in db["video_views"].find().limit(3):
    pprint(doc)
    print("\n" + "-"*80 + "\n")


Primeros 3 documentos de 'video_views':

{'_id': ObjectId('683dfda82d64dba076265475'),
 'client': {'client_id': 'tg', 'client_name': 'Toon Goggles'},
 'date': {'date_code': 20250509, 'full_date': '2025-05-09'},
 'device': {'device_type_id': 'tv', 'device_type_name': 'TV'},
 'location': {'country_code': 'US', 'country_name': 'United States'},
 'media': {'category_name': 'N/A',
           'media_id': 284074,
           'media_name': 'Suzy and The Pothole',
           'video_type': 'Episode'},
 'partner': {'partner_id': 'viziosmartcast', 'partner_name': 'Vizio Smartcast'},
 'platform': {'platform_id': 'html5', 'platform_name': 'HTML5'},
 'seconds_elapsed': 358,
 'user_id': '4c8f120a1d08d1d94ec21dabbc0b3549f26ab431'}

--------------------------------------------------------------------------------

{'_id': ObjectId('683dfda82d64dba076265476'),
 'client': {'client_id': 'tg', 'client_name': 'Toon Goggles'},
 'date': {'date_code': 20250502, 'full_date': '2025-05-02'},
 'device': {'device_type

**Creacion de Indices para mejorar la performance de las consultas que realizaremos en nuestra base de datos no relacional**

In [ ]:
# Índice por fecha (para filtrar por rango temporal en múltiples consultas)
db.video_views.create_index("date.date_code")
# Acelera búsquedas como: videos vistos en los últimos 7 o 30 días

# Índice compuesto por nombre y tipo de contenido (para top contenidos por tiempo visto)
db.video_views.create_index([
    ("media.media_name", 1),
    ("media.video_type", 1)
])
# Mejora performance al agrupar o filtrar por media_name y video_type

# Índice compuesto por cliente y fecha (para ver tiempo total por cliente en cierta fecha o periodo)
db.video_views.create_index([
    ("client.client_name", 1),
    ("date.date_code", 1)
])
# Útil para agrupar por cliente y filtrar por fechas recientes

# Índice por cliente y plataforma (para análisis conjunto de cliente y tipo de plataforma)
db.video_views.create_index([
    ("client.client_name", 1),
    ("platform.platform_name", 1)
])
# Acelera consultas como: tiempo total visto por cliente y plataforma

# Índice por cliente y título (para top títulos por cliente)
db.video_views.create_index([
    ("client.client_name", 1),
    ("media.media_name", 1)
])
# Útil para identificar qué títulos son más vistos por cada cliente


'client.client_name_1_media.media_name_1'

In [ ]:
#ver los indices que creamos para confirmar que hayan quedado bien//devuelve diccionario con clave (nombre de los indices) y valores (tiene los campos indezados y el tipo)

from pprint import pprint

pprint(db.video_views.index_information())


{'_id_': {'key': [('_id', 1)], 'v': 2},
 'client.client_name_1_date.date_code_1': {'key': [('client.client_name', 1),
                                                   ('date.date_code', 1)],
                                           'v': 2},
 'client.client_name_1_media.media_name_1': {'key': [('client.client_name', 1),
                                                     ('media.media_name', 1)],
                                             'v': 2},
 'client.client_name_1_platform.platform_name_1': {'key': [('client.client_name',
                                                            1),
                                                           ('platform.platform_name',
                                                            1)],
                                                   'v': 2},
 'date.date_code_1': {'key': [('date.date_code', 1)], 'v': 2},
 'media.media_name_1_media.video_type_1': {'key': [('media.media_name', 1),
                                              

### **Coleccion media**

In [ ]:
# Paso 1: Leer todos los documentos desde dim_media
media_docs = []

for doc in db["dim_media"].find({}):
    # Extraer solo los campos relevantes
    media_docs.append({
        "media_id": doc.get("media_id"),
        "media_name": doc.get("media_name"),
        "category_name": doc.get("category_name"),
        "video_type": doc.get("video_type")
    })

# Paso 2: Crear o reemplazar la colección media
db["media"].drop()  # Elimina si ya existía
db["media"].insert_many(media_docs)

print("Colección 'media' creada exitosamente.")


Colección 'media' creada exitosamente.


In [ ]:
#Verifico
from pprint import pprint

print("Primeros documentos en 'media':")
for doc in db["media"].find().limit(3):
    pprint(doc)


Primeros documentos en 'media':
{'_id': ObjectId('683e00f52d64dba076299da8'),
 'category_name': 'Unknown',
 'media_id': -99,
 'media_name': 'Unknown',
 'video_type': 'Unknown'}
{'_id': ObjectId('683e00f52d64dba076299da9'),
 'category_name': 'Drama',
 'media_id': 464,
 'media_name': 'MSROUT1',
 'video_type': 'Feature'}
{'_id': ObjectId('683e00f52d64dba076299daa'),
 'category_name': 'Comedia',
 'media_id': 458,
 'media_name': 'MSGIRL1',
 'video_type': 'Feature'}


**Creacion de indices para acelerar consultas**

In [ ]:
# Buscar contenido por nombre
db.media.create_index("media_name")

# Buscar por categoria
db.media.create_index("category_name")

# Combinacion para filtros + complejos
db.media.create_index([("video_type", 1), ("category_name", 1)])


'video_type_1_category_name_1'

In [ ]:
#ver los indices que creamos para confirmar que hayan quedado bien//devuelve diccionario con clave (nombre de los indices) y valores (tiene los campos indezados y el tipo)

from pprint import pprint

pprint(db.media.index_information())


### **Coleccion users**

usaremos la fact_vod_uu_details para obtener uuid_hashvalue (ID unico de cada usuario) y dim_country para obtener el pais de cada usuario y dim_device para obtener el o los dispositivos a traves de los cuales utilizan el servicio.

In [ ]:
#db["users"].drop()

Resumen: Creación de la colección users a partir de fact_vod_uu_details
Este código crea una nueva colección en MongoDB llamada users, donde cada documento representa a un usuario único identificado por el campo uuid_hashvalue. Para cada usuario, se recopilan:

- Todos los códigos de país distintos desde los que se ha conectado (countries)

- Todos los tipos de dispositivos distintos que ha utilizado (devices)

A diferencia de la colección video_views, que transformaba cada documento de forma individual usando bucles en Python, este caso requiere agrupar múltiples registros en uno solo por usuario. Por eso utilizamos la etapa de agregación (aggregation pipeline) de MongoDB, que es más eficiente para:

- Agrupar los datos directamente en el servidor

- Eliminar duplicados automáticamente usando $addToSet

- Generar resultados más limpios y rápidos, especialmente con grandes volúmenes de datos

- Este enfoque es más escalable y performante que construir los documentos manualmente con Python.

In [ ]:
# Nos conectamos nuevamente a MongoDB (por si hubo una desconexión anterior)
from pymongo import MongoClient

# Parámetros de conexión
usuario = "lab_user"
password = "bdnr2025"
host = "bdnr-2025-auriello-moli.u17itbj.mongodb.net"
db_name = "tarea_final"

# URI de conexión a MongoDB Atlas
uri = f"mongodb+srv://{usuario}:{password}@{host}/{db_name}?retryWrites=true&w=majority&appName=bdnr-2025-auriello-molina"

# Crear cliente y acceder a la base de datos
client = MongoClient(uri)
db = client[db_name]

# Eliminamos la colección 'users' si ya existe, para evitar duplicados al volver a ejecutar
db["users"].drop()

# Definimos el pipeline de agregación
# Paso 1: Agrupar todos los registros por usuario (uuid_hashvalue)
# Paso 2: Usar $addToSet para reunir valores únicos de país y tipo de dispositivo
# Paso 3: Proyectar el resultado final con formato deseado (cambiando _id a user_id)
pipeline = [
    {
        "$group": {
            "_id": "$uuid_hashvalue",                     # Agrupar por usuario
            "countries": {"$addToSet": "$country_code"},  # Lista de países únicos
            "devices": {"$addToSet": "$device_type_name"} # Lista de dispositivos únicos
        }
    },
    {
        "$project": {
            "_id": 0,             # Excluir el campo _id original
            "user_id": "$_id",    # Renombrar el campo de agrupamiento a user_id
            "countries": 1,       # Incluir campo countries
            "devices": 1          # Incluir campo devices
        }
    }
]

# Ejecutamos la agregación sobre la colección original
user_docs = list(db["fact_vod_uu_details"].aggregate(pipeline))

# Mostramos cuántos documentos se generaron
print(f"Total de documentos de usuario a insertar: {len(user_docs)}")

# Insertamos los documentos transformados en la nueva colección 'users'
if user_docs:
    db["users"].insert_many(user_docs)
    print("La colección 'users' se ha creado exitosamente.")
else:
    print("No se encontraron documentos de usuario para insertar.")


Total de documentos de usuario a insertar: 34769
La colección 'users' se ha creado exitosamente.


In [ ]:
from pprint import pprint

print("Primeros documentos en 'users':")
for doc in db["users"].find().limit(3):
    pprint(doc)


Primeros documentos en 'users':
{'_id': ObjectId('683f337fba2c293516e0b38a'),
 'countries': ['CM'],
 'devices': ['TV'],
 'user_id': '437b0a3476be144dd7796736a27ade8b1125de56'}
{'_id': ObjectId('683f337fba2c293516e0b38b'),
 'countries': ['VE'],
 'devices': ['TV'],
 'user_id': '3a316754b4bad1bbc5e6402b4f87863812bee98b'}
{'_id': ObjectId('683f337fba2c293516e0b38c'),
 'countries': ['US'],
 'devices': ['TV'],
 'user_id': '8f2c09ec8166f85ab2860aa0fd19d65b3e41db45'}


Creacion de indices para acelerar consultas

In [ ]:
# Buscar usuario por ID (uuid_hashvalue)
db.users.create_index("user_id")

# Filtrar por país
db.users.create_index("countries")  # array field, para queries tipo $in

# Buscar por tipo de dispositivo
db.users.create_index("devices")  # tambien array


'devices_1'

In [ ]:
# Verificar los índices creados en la colección 'users'
index_info = db["users"].index_information()

# Mostrar los índices en forma legible
print("Índices actuales en la colección 'users':")
for index_name, index_data in index_info.items():
    print(f"- Nombre del índice: {index_name}")
    print(f"  Definición del índice: {index_data['key']}")
    print()


Índices actuales en la colección 'users':
- Nombre del índice: _id_
  Definición del índice: [('_id', 1)]

- Nombre del índice: user_id_1
  Definición del índice: [('user_id', 1)]

- Nombre del índice: countries_1
  Definición del índice: [('countries', 1)]

- Nombre del índice: devices_1
  Definición del índice: [('devices', 1)]



# Consultas en MongoDB a nuestra base de datos no relacional

Query Original en SQL:
```
SELECT
    dm.video_type,
    dm.media_name,
    SUM(fvd.seconds_elapsed) AS total_seconds
FROM
    fact_vod_uu_details fvd
JOIN dim_media dm ON fvd.dim_media_id = dm.id
JOIN dim_date dd ON fvd.dim_date_code = dd.date_code
WHERE
    dd.full_date BETWEEN '20250501' AND '20250510'
    AND dm.client_id = 'tg'
    AND dm.video_type != 'Linear'
GROUP BY
    dm.video_type, dm.media_name
ORDER BY
    total_seconds DESC
LIMIT 10

```



In [2]:
# Consulta 1: Aggregate total_seconds por video_type y media_name
pipeline = [
    {
        "$match": {
            "client.client_id": "tg"
        }
    },
    {
        "$group": {
            "_id": "$media.media_name",
            "total_seconds": { "$sum": "$seconds_elapsed" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "media_name": "$_id",
            "total_seconds": 1
        }
    },
    {
        "$sort": {
            "total_seconds": -1
        }
    },
    {
        "$limit": 10
    }
]

# Ejecutamos el pipeline
results = db.video_views.aggregate(pipeline)

# Paso 2: Mostramos los resultados
print("Top 10 contenidos por tiempo visto:")
for doc in results:
    media_name = doc["media_name"]
    total_seconds = doc["total_seconds"]
    print(f"{media_name} => {total_seconds} segundos")

Top 10 contenidos por tiempo visto:
My Little Pony Equestria Girls: Rainbow Rocks => 3542125 segundos
Fire-Breathing Magic Dinosaur => 876877 segundos
Merry-go-round => 867021 segundos
Diamond Dog Caper => 830416 segundos
Beat the Raccoon => 636846 segundos
Pirates of Love => 614098 segundos
Magic wand => 588624 segundos
End of Year Special - Bubu and the Little Owls => 571157 segundos
The Missing Hero Crystal => 553976 segundos
Holidays Forever Compilation => 551834 segundos


Consulta Original:



```
SELECT
    dc.client_name,
    SUM(fvd.seconds_elapsed) AS total_seconds
FROM
    prod.fact_vod_uu_details fvd
JOIN prod.dim_client AS dc ON fvd.dim_client_id = dc.id
JOIN prod.dim_date AS dd ON fvd.dim_date_code = dd.date_code
INNER JOIN prod.dim_media AS m ON m.id=fvd.dim_media_id
WHERE
    dd.full_date BETWEEN '20250501' and '20250510'
    AND dc.client_id='tg'
    AND m.video_type!='Linear'
GROUP BY
    dc.client_name
ORDER BY
    total_seconds DESC
```



In [3]:
##Consulta2
pipeline = [
    {
        "$match": {
            "client.client_id": "tg"
        }
    },
    {
        "$group": {
            "_id": "$client.client_name",
            "total_seconds": { "$sum": "$seconds_elapsed" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "client_name": "$_id",
            "total_seconds": 1
        }
    },
    {
        "$sort": {
            "total_seconds": -1
        }
    },
    {
        "$limit": 10
    }
]

# Ejecutamos el pipeline
results = db.video_views.aggregate(pipeline)

# Paso 2: Mostramos los resultados
print("Top 10 clientes por tiempo visto:")
for doc in results:
    client_name = doc["client_name"]
    total_seconds = doc["total_seconds"]
    print(f"{client_name} => {total_seconds} segundos")

Top 10 clientes por tiempo visto:
Toon Goggles => 92363525 segundos


Consulta Original SQL:



```
SELECT
    dd.full_date,
    COUNT(DISTINCT uuid_hashvalue) AS total_users
FROM
    prod.fact_vod_uu_details AS fvd

INNER JOIN prod.dim_date dd ON fvd.dim_date_code = dd.date_code
INNER JOIN prod.dim_client AS c ON c.id=fvd.dim_client_id
INNER JOIN prod.dim_media AS m ON m.id=fvd.dim_media_id

WHERE
dd.full_date BETWEEN '20250501' AND '20250510'
AND c.client_id='tg'
AND m.video_type!='Linear'

GROUP BY
    dd.full_date
ORDER BY
    dd.full_date ASC;
```



In [4]:
#Consulta 3
pipeline = [
    {
        "$match": {
            "client_id": "tg",
            "video_type": { "$ne": "Linear" }
        }
    },
    # Agrupamos por full_date y uuid_hashvalue para obtener combinaciones únicas
    {
        "$group": {
            "_id": {
                "full_date": "$full_date",
                "uuid": "$uuid_hashvalue"
            }
        }
    },
    # Agrupamos por full_date y contamos los uuid únicos
    {
        "$group": {
            "_id": "$_id.full_date",
            "unique_users": { "$sum": 1 }
        }
    },
    # Formateamos la salida
    {
        "$project": {
            "_id": 0,
            "full_date": "$_id",
            "unique_users": 1
        }
    },
    # Orden cronológico
    {
        "$sort": {
            "full_date": 1
        }
    }
]

results = db.fact_vod_uu_details.aggregate(pipeline)

for doc in results:
    print(f"{doc['full_date']} => {doc['unique_users']} unique users")


2025-05-01 => 5729 unique users
2025-05-02 => 4530 unique users
2025-05-03 => 4866 unique users
2025-05-04 => 4546 unique users
2025-05-05 => 3957 unique users
2025-05-06 => 3914 unique users
2025-05-07 => 4009 unique users
2025-05-08 => 4092 unique users
2025-05-09 => 4189 unique users
2025-05-10 => 4843 unique users


Consulta Original SQL



```
SELECT
    dc.client_name,
    dp.partner_name,
    SUM(fvd.seconds_elapsed) AS total_seconds
FROM
    prod.fact_vod_uu_details AS fvd

INNER JOIN prod.dim_date AS dd ON dd.date_code=fvd.dim_date_code
INNER JOIN prod.dim_client dc ON fvd.dim_client_id = dc.id
INNER JOIN prod.dim_media AS m ON m.id=fvd.dim_media_id
INNER JOIN prod.dim_partner dp ON fvd.dim_partner_id = dp.id

WHERE
dd.full_date BETWEEN '20250501' AND '20250510'
AND dc.client_id='tg'
AND m.video_type!='Linear'
```



In [ ]:
#Consulta 4
pipeline = [
    {
        "$group": {
            "_id": {
                "client_name": "$client_name",
                "partner_name": "$partner_name"
            },
            "total_seconds": { "$sum": "$seconds_watched" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "client_name": "$_id.client_name",
            "partner_name": "$_id.partner_name",
            "total_seconds": 1
        }
    },
    {
        "$sort": {
            "total_seconds": -1
        }
    },
    {
        "$limit": 10
    }
]


results = db.fact_vod_uu_details.aggregate(pipeline)

for doc in results:
    print(f"{doc['client_name']} => {doc['partner_name']}, {doc['total_seconds']}")

Toon Goggles => Hisense, 27459163
Toon Goggles => Roku, 13100312
Toon Goggles => Vizio Smartcast, 9557878
Toon Goggles => TCL, 9469610
Toon Goggles => M Star, 7468756
Toon Goggles => General Android, 4024805
Toon Goggles => Philips, 3668155
Toon Goggles => Unknown, 2898904
Toon Goggles => General HTML5, 1845031
Toon Goggles => Hisense Legacy, 1665119
